In [1]:
import os 
import pandas as pd
from model import AfrikaPOS

/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_DIR = os.path.join('..', 'data_source', 'masakhane-pos', 'data')
LANGS = os.listdir(DATA_DIR)

BASE_MODEL = 'bert-base-uncased'
NUM_HIDDENS = 2
NUM_ATTENTIONS = 12

exec_counter = 1

In [3]:
def train_model_for_language(data_dir, out_dir):
    model = AfrikaPOS(data_dir, NUM_HIDDENS, NUM_ATTENTIONS, BASE_MODEL)
    model.train_model(out_dir, num_train_epochs=50)
    return model

In [4]:
results = {}
model_sizes = {}
for lang in LANGS:
    print("\n\n**********************\nExecuting to language:", lang, "\n**********************")
    out_dir = os.path.join("models", lang)
    data_dir = os.path.join(DATA_DIR, lang)
    model = train_model_for_language(data_dir, out_dir)
    test_perfs = model.evaluate_on_test()
    results[lang] = test_perfs

    n_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
    model_sizes[lang] = {'n_params': n_params, 'vocab_size': model.tokenizer.vocab_size}



**********************
Executing to language: ibo 
**********************



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  2%|▏         | 51/2550 [00:06<05:52,  7.09it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control t

{'eval_loss': 0.5978075861930847, 'eval_precision': 0.596370847874501, 'eval_recall': 0.5660259089009612, 'eval_f1': 0.5389260036821545, 'eval_accuracy': 0.5660259089009612, 'eval_runtime': 0.6599, 'eval_samples_per_second': 242.443, 'eval_steps_per_second': 15.153, 'epoch': 1.0}


  4%|▍         | 102/2550 [00:12<04:15,  9.59it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

{'eval_loss': 0.4930618405342102, 'eval_precision': 0.6600805432136099, 'eval_recall': 0.6391558712912662, 'eval_f1': 0.6348260752378692, 'eval_accuracy': 0.6391558712912662, 'eval_runtime': 0.2612, 'eval_samples_per_second': 612.572, 'eval_steps_per_second': 38.286, 'epoch': 2.0}


...

100%|██████████| 2200/2200 [07:28<00:00,  7.59it/s]

{'eval_loss': 0.3553053140640259, 'eval_precision': 0.8013886151546205, 'eval_recall': 0.8001047943411056, 'eval_f1': 0.7969396008472286, 'eval_accuracy': 0.8001047943411056, 'eval_runtime': 0.2947, 'eval_samples_per_second': 468.325, 'eval_steps_per_second': 30.543, 'epoch': 50.0}


100%|██████████| 2200/2200 [07:29<00:00,  4.90it/s]


{'train_runtime': 449.0549, 'train_samples_per_second': 77.162, 'train_steps_per_second': 4.899, 'train_loss': 0.17021876811981201, 'epoch': 50.0}


 97%|█████████▋| 34/35 [00:06<00:00, 10.45it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

In [13]:
model_sizes_df = pd.DataFrame(model_sizes)
model_sizes_df.to_csv(f"results/model_size{exec_counter}.csv")
model_sizes_df

,ibo,sna,bbj,wol,bam,ewe,mos,nya,lug,xho,kin,pcm,twi,fon,zul,yor,hau,swa
n_params,18627857,20805905,17008913,18458897,18425873,18173969,18100241,20052497,19953425,23429393,20603921,19123217,19255313,16715537,22204433,17350673,18281489,20317457
vocab_size,5264,8100,3156,5044,5001,4673,4577,7119,6990,11516,7837,5909,6081,2774,9921,3601,4813,7464


In [12]:
result_df = pd.concat(results)
result_df.to_csv(f"results/results{exec_counter}.csv")
result_df

ADJ          ADP         ADV          AUX       CCONJ  \
ibo precision     0.726804     0.673768    0.100295     0.296181    0.464286   
    recall        0.888655     0.710175    0.348718     0.763052    0.534884   
    f1-score      0.799622     0.691493    0.155785     0.426727    0.497091   
    support     476.000000  1425.000000  195.000000   498.000000  559.000000   
sna precision     0.612245     0.731670    0.444805     0.823188    0.579208   
...                    ...          ...         ...          ...         ...   
hau support     492.000000   511.000000  396.000000  1137.000000  392.000000   
swa precision     0.820354     0.882653    0.681979     0.698795    0.612482   
    recall        0.917322     0.659048    0.451991     0.660969    0.988290   
    f1-score      0.866132     0.754635    0.543662     0.679356    0.756272   
    support    3181.000000  1050.000000  854.000000   351.000000  427.000000   

                       DET  INTJ         NOUN         NUM    PART  \
ibo precision     0.518896   0.0     0.881446    0.767568     0.0   
    recall        0.740582   0.0     0.710439    0.663551     0.0   
    f1-score      0.610229   0.0     0.786757    0.711779     0.0   
    support    1168.000000   1.0  7242.000000  428.000000     3.0   
sna precision     0.752203   NaN     0.797116    0.867841     0.0   
...                    ...   ...          ...         ...     ...   
hau support     186.000000   3.0  3676.000000  544.000000  2018.0   
swa precision     0.000000   NaN     0.825054    1.000000     0.0   
    recall        0.000000   NaN     0.870355    0.674528     0.0   
    f1-score      0.000000   NaN     0.847099    0.805634     0.0   
    support      16.000000   NaN  4790.000000  212.000000   100.0   

                      PRON        PROPN        PUNCT       SCONJ       SYM  \
ibo precision     0.654392     0.859964     0.993435    0.074074  0.000000   
    recall        0.925926     0.390701     0.991266    0.007435  0.000000   
    f1-score      0.766831     0.537297     0.992350    0.013514  0.000000   
    support    1593.000000  1226.000000  1832.000000  807.000000  7.000000   
sna precision     0.707865     0.727023     0.977953    0.917582  0.846154   
...                    ...          ...          ...         ...       ...   
hau support     475.000000  2002.000000  1274.000000  269.000000       NaN   
swa precision     0.666667     0.711812     1.000000    0.744726       NaN   
    recall        0.179104     0.727100     0.965170    0.559429       NaN   
    f1-score      0.282353     0.719375     0.982276    0.638914       NaN   
    support     134.000000  1583.000000  1292.000000  631.000000       NaN   

                      VERB  accuracy     macro avg  weighted avg      X  
ibo precision     0.758260  0.719441      0.485586      0.748612    NaN  
    recall        0.787993  0.719441      0.528961      0.719441    NaN  
    f1-score      0.772841  0.719441      0.485145      0.719015    NaN  
    support    4514.000000  0.719441  21974.000000  21974.000000    NaN  
sna precision     0.627143  0.750258      0.694133      0.747993    NaN  
...                    ...       ...           ...           ...    ...  
hau support    1877.000000  0.814354  15438.000000  15438.000000  186.0  
swa precision     0.578046  0.786861      0.614838      0.784306    0.0  
    recall        0.627410  0.786861      0.552048      0.786861    0.0  
    f1-score      0.601717  0.786861      0.565162      0.779156    0.0  
    support    1452.000000  0.786861  16074.000000  16074.000000    1.0  

[72 rows x 20 columns]

In [4]:
from utils import get_tokenizer_training_corpus

lang = 'ewe'
data_dir = os.path.join(DATA_DIR, lang)
data = [" ".join(s) for s in get_tokenizer_training_corpus(data_dir)]
data

['CEMAC : Dukplɔla siwo le hatsotso sia me wɔ ɖoɖo ku ɖe nutoame ƒe ganyawo ŋu le tsɔme .',
 'Dukplɔla kple dudɔnunɔla tatɔ siwo le Titina Afrika duwo ƒe Ƒuƒoƒo hena Gadzimɔnunyawo ( CEMAC ) me do go nyitsɔ kuɖagbe le aɖaŋuɖemɔtakpekpe aɖe me .',
 'Gododoa ƒe tanyagãe nye : Tamebubu du siwo siwo le hatsotsoa me la ƒe ganyawo ŋu .',
 'CAN 2022 : Asamoah Gyan le Maroc vɔm .',
 'Ghana bɔlƒola xɔŋkɔa , Asamoah Gyan ƒo nu hã kuɖe tsatsiã si wowɔ hena ƒuƒoƒomenɔnɔ le CAN 2022 me eye wotrɔ ŋku tɔxɛ ɖe ƒuƒoƒo si wokpɔ be eyae nye sesẽtɔ wu la ŋu .',
 "CAN 2022 : Samuel Eto'o ƒo nu ku ɖe Cameroun ƒe dzɔgbe nyuiwo ŋu .",
 'Cameroun sie nye du si le CAN 2022 ƒe zime la , le ƒuƒoƒo me kple Burkina , Ethiopie kpakple Cap .',
 'Ƒoƒoƒo sia , Dzata ( Lions ) ate ŋu kpe edzi faa .',
 "Gake , Samuel Eto'o be naneke mekpɔ dzɔ haɖe ke o .",
 'Burkina Faso dziɖuɖua ɖee ɖe go be wowu hlɔ̃dola 58 aɖewo le aʋahoho ɖe wo ŋu le Arbinda kple Gorgadji dome , wɔna sia na ame gbɔlo kple Kpeɖeŋunala kpakple gbad

In [8]:
model.predict(data[1])

['VERB',
 'NOUN',
 'VERB',
 'ADJ',
 'VERB',
 'VERB',
 'VERB',
 'AUX',
 'PROPN',
 'NOUN',
 'NOUN',
 'VERB',
 'ADJ',
 'NOUN',
 'PROPN',
 'VERB',
 'PROPN',
 'NOUN',
 'VERB',
 'VERB',
 'NOUN',
 'NOUN',
 'VERB',
 'PUNCT',
 'NOUN',
 'NOUN']